In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random, getopt, os, sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from Lamp.AttrDict.AttrDict import *
from Lamp.Model.Dataloader import *
from Lamp.Model.BaseModel import *

In [7]:
def load_config(cfg_path):
    """  """
    if os.path.splitext(cfg_path)[-1] == '.json':
        return AttrDict.from_json_path(cfg_path)
    elif os.path.splitext(cfg_path)[-1] in ['.yaml', '.yml']:
        return AttrDict.from_yaml_path(cfg_path)
    else:
        raise ValueError(f"Unsupported config file format. Only '.json', '.yaml' and '.yml' files are supported.")

In [1]:
ifile = "../Models/config/MAR_RESNET34_PADDED_256_ALL.yaml"

In [2]:
inputs = load_config(ifile)


NameError: name 'load_config' is not defined

In [5]:
root_path = os.path.abspath(os.path.join(os.getcwd(), '../'))
path_load_data = f"{root_path}/{inputs.LoadPath}"
path_model = f"{root_path}/{inputs.PathSave}/{inputs.ModelName}"

model_name = f"model_all.pt"
save_model_path = f"{path_model}/{model_name}"

dataframe = pd.read_csv(path_load_data,index_col=0)

# Train Test Split
_, test_dataframe = train_test_split(dataframe, test_size=(1 - inputs.TrainTestSplit),stratify=dataframe['Label'], random_state=inputs.Seed)

# Reset Index
test_dataframe = test_dataframe.reset_index(drop=True)

# Samples
test_dataframe = test_dataframe.groupby('Label').sample(20,replace=True,random_state=inputs.Seed).reset_index(drop=True)

NameError: name 'inputs' is not defined

In [51]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = resnet(layers=layers,channels=channels,num_classes=classes)

classifier = Classifier(
                net=net, 
                device=device
                )

classifier.load(save_model_path)

pred, label = classifier.predict(test_dataloader)

In [57]:
if os.path.isfile('comparison_data.csv'):
    comparison_data = pd.read_csv('comparison_data.csv',index_col=0)
else :
    comparison_data = pd.DataFrame()
    comparison_data['Paths'] = test_dataframe['Paths']
    comparison_data['Label'] = test_dataframe['Label']

comparison_data[f'{inputs.ModelName}_best_model'] = pred
comparison_data[f'{inputs.ModelName}_baseline_model'] = pred

comparison_data.to_csv('comparison_data.csv')